We are assuming that bulbs of wattage less than or equal to 90 are used for local/collector roads and those that are greater than 90 are used for major/local roads, given the streetlight city regulations, we can calculate the difference between what is considered a high, medium, and low level of illuminance


In [ ]:
import re
#Use the power of the light to guess how much of an area they have to light up
def getarea(wattage):
    if float(wattage)<=90.0:
        return 2120.0
    return 2905.0

flux = 10.76391 # a constant used to calculate lux
efficacy = 55.6 # the efficacy level of induction lights





Here we open the file to parse out the different columns, and create the columns for our analysis
"lux_val": Calculated luminosity
"med_wat": Estimated medium luminosity, -6 Lux away from lux_val
"low_wat": Estimated low luminosity, -12 Lux away from lux_val
"full_cost": Price being paid now, $0.15 per kWh
"med_cost": Price adjusted for med_wat
"low_cost": Price adjusted for low_wat

In [ ]:
 fi = open("StreetLight_LocationsClean.csv",'r')
outfi = open("streetlight_locations_dataLux.csv",'w')

line = fi.readline()
line=re.sub('\n',',',line)
outfi.write(line+"\"lux_val\","+"\"med_wat\","+"\"low_wat\","+"\"full_cost\","+"\"med_cost\","+"\"low_cost\"\n")
line = fi.readline()



Given the formula...
    lx = 10.76391 × W × (lm/W) / ft2
    where lx is the illuminance, W is the wattage, lm/W is the luminous efficacy, and ft2 is the area in square feet

...we can calculate what the current luminosity is at right now, and according to weather and traffic conditions, we can adjust the luminosity and find the new wattage. This is how we are calculating cost
 

In [ ]:
while line:
    line=re.sub('\n','',line)
    parts = line.split("\"")
    if parts[3]=="IND" and parts[5]!=0.0 and parts[5]!="":
        #if the light type is induction then find lux and possible dimm values
        lux=(float(parts[5])*efficacy*flux)/(getarea(parts[5]))
        med=lux-6
        med=getarea(parts[5])*med/(efficacy*flux)
        low=lux-12
        low=getarea(parts[5])*low/(efficacy*flux)
    else:
        #if the light is not induction then we can't fint those values
        lux=0
        med=0
        low=0
        

In [ ]:
        
    #output new data
    outfi.write("\""+parts[1]+"\",")
    outfi.write("\""+parts[3]+"\",")
    outfi.write("\""+parts[5]+"\",")
    outfi.write("\""+parts[7]+"\",")
    outfi.write("\""+parts[9]+"\",")
    outfi.write("\""+parts[11]+"\",")
    outfi.write("\""+str(lux)+"\",")
    outfi.write("\""+str(med)+"\",")
    outfi.write("\""+str(low)+"\",")
    if parts[5]=='':
        outfi.write("\"0\",")
    else:
                outfi.write("\""+str((float(parts[5])/1000.0)*11.0*0.15)+"\",")

    #11.0 for 11 hours of operation, 0.15 because California charges $0.15 per kWh
    outfi.write("\""+str((med/1000.0)*11.0*0.15)+"\",")
    outfi.write("\""+str((low/1000.0)*11.0*0.15)+"\"\n")
    line=fi.readline()
fi.close()
outfi.close()